In [1]:
import requests
import pandas as pd
import time
from datetime import date, datetime, timedelta

In [2]:
forest_data = pd.read_csv(r'')

In [3]:
forest_data.head()

,Unnamed: 0,area_type_cd,site_type_cd,moisture_cd,degradation_cd,soil_subtype_cd,plant_comm_cd,stand_struct_cd,sub_area,veg_cover_cd,forest_id,avg_lat,avg_lon,tree_species,area
0,0,D-STAN,BŚW,Ś,N2,RDw,NaN,DRZEW,9.20,MSZC,101025863,53.811817,23.275983,SO,9.20
1,1,D-STAN,OLJ,ŁZ,N2,Mt,NaN,KDO,8.44,ZAD,101027565,53.793429,23.136016,OL,8.44
2,2,D-STAN,BŚW,Ś,N2,RDb,NaN,DRZEW,8.12,ZAD,101028430,53.710679,23.036625,SO,8.12
3,3,D-STAN,LMB,BO,N2,Mt,NaN,DRZEW,5.35,ZIEL,101008890,53.773599,23.273369,?W,5.35
4,4,D-STAN,BŚW,Ś,N2,RDb,NaN,DRZEW,14.12,ZAD,101009062,53.776887,23.304634,SO,14.12


In [4]:
forest_data.shape

(366609, 15)

In [5]:
forest_data = forest_data[forest_data.sub_area > 10]
forest_data.shape

(100430, 15)

In [6]:
forest_data.head()

,Unnamed: 0,area_type_cd,site_type_cd,moisture_cd,degradation_cd,soil_subtype_cd,plant_comm_cd,stand_struct_cd,sub_area,veg_cover_cd,forest_id,avg_lat,avg_lon,tree_species,area
4,4,D-STAN,BŚW,Ś,N2,RDb,NaN,DRZEW,14.12,ZAD,101009062,53.776887,23.304634,SO,14.12
10,10,D-STAN,BB,BM,N1,Tw,NaN,DRZEW,13.06,ZAD,101007586,53.807356,23.287168,SO,13.06
12,12,D-STAN,BŚW,SŚ,N2,RDb,NaN,DRZEW,13.86,MSZC,101002690,53.845842,23.191385,SO,13.86
13,13,D-STAN,OL,BM,N2,Tn,NaN,DRZEW,13.01,ZAD,101008034,53.802518,23.321325,BRZ,13.01
14,14,D-STAN,BŚW,Ś,N2,RDb,NaN,DRZEW,14.02,MSZC,101011496,53.787320,23.084254,SO,14.02


In [ ]:
weather_data = pd.DataFrame(columns=["date","forest_id","latitude","longitude","tree_species","soil_subtype_cd","max_temp","min_temp","precipitation","snowfall","humidity"])

y=0

for index, row in forest_data.iterrows():
    try:
        date_today = date.today()
        date = date_today - timedelta(days=31)
        date_1_day_ago = date - timedelta(days=1)
        date_14_days_ago = date - timedelta(days=14)

        forest_id = row["forest_id"]
        latitude = row["avg_lat"]
        longitude = row["avg_lon"]

        tree_species = row["tree_species"]
        soil_type = row["soil_subtype_cd"]
        
        request_string = f"""
        https://api.open-meteo.com/v1/forecast?latitude={latitude}
        &longitude={longitude}&hourly=relativehumidity_2m,
        soil_temperature_6cm,
        soil_moisture_3_9cm&daily=temperature_2m_max,
        temperature_2m_min,rain_sum,snowfall_sum&timezone=Europe%2FBerlin&start_date={date_14_days_ago}
        &end_date={date_1_day_ago}"""

        result = requests.get(request_string)
        weather = result.json()
        
        max_temp = weather["daily"]["temperature_2m_max"]
        min_temp = weather["daily"]["temperature_2m_min"]
        precipitation = weather["daily"]["rain_sum"]
        snowfall = weather["daily"]["snowfall_sum"]
        humidity = weather["hourly"]["relativehumidity_2m"]

        row = [date, forest_id, latitude, longitude, tree_species, soil_type, max_temp, min_temp, precipitation, snowfall, humidity]
        weather_data.loc[y] = row
        
        y=y+1
                
        if y%1000==0:
                print(f'{y} rows completed.')
                time.sleep(1)
    except:
        pass

1000 rows completed.
2000 rows completed.
3000 rows completed.
4000 rows completed.
5000 rows completed.
6000 rows completed.
7000 rows completed.
8000 rows completed.
9000 rows completed.
10000 rows completed.
11000 rows completed.
12000 rows completed.
13000 rows completed.
14000 rows completed.
15000 rows completed.
16000 rows completed.
17000 rows completed.
18000 rows completed.
19000 rows completed.
20000 rows completed.
21000 rows completed.
22000 rows completed.
23000 rows completed.
24000 rows completed.
25000 rows completed.
26000 rows completed.
27000 rows completed.
28000 rows completed.
29000 rows completed.
30000 rows completed.
31000 rows completed.
32000 rows completed.
33000 rows completed.
34000 rows completed.
35000 rows completed.
36000 rows completed.
37000 rows completed.
38000 rows completed.
39000 rows completed.
40000 rows completed.
41000 rows completed.
42000 rows completed.
43000 rows completed.
44000 rows completed.
45000 rows completed.
46000 rows complete

In [ ]:
weather_data.head()

In [ ]:
weather_data.shape

In [ ]:
weather_data.to_csv(r"C:\Users\julia\Desktop\Mushroom Project\data\geodata\compiled\weather_data.csv")